The following packages need to be downloaded:

In [11]:
data.clear_download_cache(None)

Astroquery,
numpy >= 1.14,
astropy (>=3.1),
requests,
keyring,
Beautiful Soup,
html5lib,
six,
curl,
pytest-astropy,
pyregion,



Run this section first to load all the prerequisite packages.

In [1]:
from astroquery.vizier import Vizier
import requests
from bs4 import BeautifulSoup
from astroquery.simbad import Simbad
import numpy as np
from astropy.table import Table, vstack
from astroquery.xmatch import XMatch
from astropy.io import ascii
from astropy import units as u
from astropy.utils import data
from astropy.coordinates import SkyCoord, get_constellation
import pandas as pd 
import kaggle
import ipython_bell
import glob, os

Could not import regions, which is required for some of the functionalities of this module.


Please run this section of the code to download the data on constellation borders. The csv file will contain the ra and dec that must be plotted (scatter) to obtain the borders of constellations

In [31]:
#First we import astroquery and remove the limit on rows.

from astroquery.vizier import Vizier
catalogue_list = Vizier.find_catalogs("Constellation")#This loads all the catalogs by the keyword "constellation"
Vizier.ROW_LIMIT = -1

catalog = Vizier.get_catalogs("VI/49")[1]#This selects the 2nd of three tables that were found by the key-word search. This table contains relevant data

catalog.remove_columns(['cst','type'])

coords = SkyCoord(catalog['RAJ2000'],catalog['DEJ2000'],unit="deg")
const = coords.get_constellation()
catalog.add_column(const,name="Constellation", index = 2)

catalog.write("constellation_borders.csv",format="csv",overwrite="True")#creates a csv file
%bell

The output file generated by the above script will be stored in your working directory. to load tables directly from the Internet using URLs. For example, download tables from Vizier catalogues in CDS format ('ascii.cds'):

Run this section to download the ra,dec of messier objects

In [9]:
#PARSING THROUGH HYPERLEDA
#will add all mags into an array
mags = []

import requests
from bs4 import BeautifulSoup
for i in range(110):
    page = requests.get("http://leda.univ-lyon1.fr/ledacat.cgi?o=M{}".format(i+1))
    soup = BeautifulSoup(page.content,'html.parser')
    if len(soup.find_all('table',attrs={"class":"datatable"}))!=0:
        table_rel = soup.find_all('table',attrs={"class":"datatable"})[1]
        index = 0
        for rows in table_rel.find_all('tr'):
            if rows.find_all('a', attrs = {"title":"Total V-magnitude"}) != []:
                            mag_value = rows.find_all('td')[1].get_text().split()[0]
                            mags.append(mag_value)
                            index = index + 1
                            break
            if rows == table_rel.find_all('tr')[-1] and index == 0:
                mags.append('-')
                break
    else:
        mags.append('-')
        continue

#NEED TO ADD THESE VALUES TO THE MESSIER TABLE

In [49]:

#Import Simbad query
from astroquery.simbad import Simbad

#the next three lines are to simplify the output table
Simbad.reset_votable_fields()
Simbad.remove_votable_fields('coordinates')
Simbad.add_votable_fields('otype(3)', 'ra(d;A;ICRS;J2000;2000)', 'dec(d;D;ICRS;J2000;2000)','flux(B)','flux(V)')#These are all the columns added to the table

result_table = Simbad.query_catalog("Messier")#This asks the SIMBAD database to list all objects from the messier catalog
#result_table['V Mag (from HyperLeda/SED)'] = mags#adding v mags
result_table['FLUX_V'].name = 'V'
result_table['OTYPE_3']=[x.decode('utf8') for x in result_table['OTYPE_3']]
result_table['MAIN_ID']=[x.decode('utf8') for x in result_table['MAIN_ID']]
result_table['RA_d_A_ICRS_J2000_2000'].name = "RAJ2000"
result_table['DEC_d_D_ICRS_J2000_2000'].name = "DEJ2000"
result_table['B-V'] = result_table['FLUX_B'] - result_table['V']
result_table.remove_column('FLUX_B')

coords = SkyCoord(result_table['RAJ2000'],result_table['DEJ2000'],unit="deg")
const = coords.get_constellation()
const_abr = coords.get_constellation(short_name = "True")
result_table.add_column(const,name="Constellation", index = 2)

otype = result_table['OTYPE_3']
internal_id = [otype[i]+'_'+const_abr[i]+'_'+'M{}'.format(i+1) for i in range(len(const))]
result_table.add_column(internal_id, name = "Internal ID Number", index = 0)

result_table.write("messier_objects.csv",format="csv",overwrite="True")#creates a csv file

Simbad.reset_votable_fields()#renders the prev changes to simbad class temporary.

NGC Catalogue

In [50]:
#CHANGE GET_CONSTELLATION EQUINOX FROM B1875 TO 2000
from astroquery.vizier import Vizier
v = Vizier(columns = ['Name','Type','mag','RA (deg)','Dec (deg)'])#Columns added to table
v.ROW_LIMIT = -1
result_table = v.get_catalogs("VII/118/ngc2000")[0]

result_table['Name'] = ['IC '+x[1:] if x[0]=='I' else 'NGC '+x for x in result_table['Name']]
result_table['Type'] = ['Gal' if x=='Gx' else 'OpC' if x=='OC' else 'GlC' if x=='Gb' else 'PN' if x=='Pl' else 'Str' if x=='*' or x=='D*' or x=='***' else '-' if x == '' or x=='-' or x=='?' else x for x in result_table['Type']]

coords = SkyCoord(result_table['_RAJ2000'],result_table['_DEJ2000'],unit="deg")
const = coords.get_constellation()
const_abr = coords.get_constellation(short_name = "True")
result_table.add_column(const,name="Constellation", index = 2)

otype = result_table['Type']
internal_id = [otype[i]+'_'+const_abr[i]+'_'+result_table['Name'][i] if otype[i]!='-' else '-'+'_'+const_abr[i]+'_'+result_table['Name'][i] for i in range(len(const))]
result_table.add_column(internal_id, name = "Internal ID Number", index = 0)

result_table.write("NGC.csv",format="csv",overwrite="True")


Hipparcos Catalogue

In [ ]:
#Part of Hipparcos catalogue below 6 mags
#from astroquery.vizier import Vizier
#v = Vizier(columns = ['HD','_RAJ2000','DECJ2000','B-V','Vmag','SpType','Plx'], column_filters = {"Vmag":"<=6"})
#v.ROW_LIMIT = -1
#v.TIMEOUT = 1000
#catalog_stars = v.get_catalogs("I/239/hip_main")[0]
#catalog_stars.write("Hipparcos_1.csv",format="csv",overwrite = "True")


In [ ]:
#Part of Hipparcos catalogue between 6 to 8 mags
#from astroquery.vizier import Vizier
#v = Vizier(columns = ['HD','_RAJ2000','DECJ2000','B-V','Vmag','SpType','Plx'], column_filters = {"Vmag":"6..8"})
#v.ROW_LIMIT = -1
#v.TIMEOUT = 1000
#catalog_stars = v.get_catalogs("I/239/hip_main")[0]
#catalog_stars.write("Hipparcos_2.csv",format="csv",overwrite = "True")

Tycho Catalogue

In [62]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','+Vmag'], column_filters = {"Vmag":"<6"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars['Vmag'].name = "V"

coords = SkyCoord(catalog_stars['_RAJ2000'],catalog_stars['_DEJ2000'],unit="deg")
const = coords.get_constellation()
const_abr = coords.get_constellation(short_name = "True")

catalog_stars.add_column(const,name="Constellation", index = 0)

internal_id = ["Str_"+const_abr[i]+"_"+f"{i+1:08d}" for i in range(len(catalog_stars['_RAJ2000']))]
catalog_stars.add_column(internal_id, name="Internal ID Number", index = 0)

catalog_stars.write("tycho_1.csv",format="csv",overwrite = "True")

last_id = len(catalog_stars['_RAJ2000'])

%bell

In [63]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','+Vmag'], column_filters = {"Vmag":"6..9"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars['Vmag'].name = "V"

coords = SkyCoord(catalog_stars['_RAJ2000'],catalog_stars['_DEJ2000'],unit="deg")
const = coords.get_constellation()
const_abr = coords.get_constellation(short_name = "True")

catalog_stars.add_column(const,name="Constellation", index = 0)

internal_id = ["Str_"+const_abr[i]+"_"+f"{i+last_id+1:08d}" for i in range(len(catalog_stars['_RAJ2000']))]
catalog_stars.add_column(internal_id, name="Internal ID Number", index = 0)

catalog_stars.write("tycho_2.csv",format="csv",overwrite = "True")

last_id = last_id +len(catalog_stars['_RAJ2000'])
%bell

In [64]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','+Vmag'], column_filters = {"Vmag":"9..10"})
v.ROW_LIMIT = 10000000
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars['Vmag'].name = "V"

coords = SkyCoord(catalog_stars['_RAJ2000'],catalog_stars['_DEJ2000'],unit="deg")
const = coords.get_constellation()
const_abr = coords.get_constellation(short_name = "True")

catalog_stars.add_column(const,name="Constellation", index = 0)

internal_id = ["Str_"+const_abr[i]+"_"+f"{i+last_id+1:08d}" for i in range(len(catalog_stars['_RAJ2000']))]
catalog_stars.add_column(internal_id, name="Internal ID Number", index = 0)

catalog_stars.write("tycho_3.csv",format="csv",overwrite = "True")
last_id = last_id +len(catalog_stars['_RAJ2000'])
%bell

In [18]:
v = Vizier(columns = ['HIP','TYC','HD','_RAJ2000','DECJ2000','B-V','+Vmag'], column_filters = {"Vmag":"10..12"})
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
catalog_stars = v.get_catalogs("I/239/tyc_main")[0]
catalog_stars['Vmag'].name = "V"

coords = SkyCoord(catalog_stars['_RAJ2000'],catalog_stars['_DEJ2000'],unit="deg")
const = coords.get_constellation()
const_abr = coords.get_constellation(short_name = "True")

catalog_stars.add_column(const,name="Constellation", index = 0)

internal_id = ["Str_"+const_abr[i]+"_"+f"{i+last_id+1:08d}" for i in range(len(catalog_stars['_RAJ2000']))]
catalog_stars.add_column(internal_id, name="Internal ID Number", index = 0)

catalog_stars.write("tycho_4.csv",format="csv",overwrite = "True")
%bell

In [68]:
name_list = []
for file in glob.glob("tycho_*.csv"):
    name_list.append(file)

name_list.sort()

t = pd.concat([pd.read_csv(i) for i in name_list])
t.to_csv('tycho.csv')
    

Gaia DR2

In [ ]:
from astroquery.gaia import Gaia
Gaia.ROW_LIMIT = 10000000000
#for column in gaiadr1_table.columns:
#    print(column.name)

job = Gaia.launch_job_async("SELECT TOP 1000 ra,dec,phot_g_mean_mag+0.01760+0.006860*bp_rp+0.1732*power((bp_rp),2) as v FROM gaiadr2.gaia_source WHERE (phot_g_mean_mag+0.01760+0.006860*bp_rp+0.1732*power((bp_rp),2) < 6)")
r = job.get_results()
#g = r['phot_g_mean_mag']
#br = r['bp_rp']
#v = g+0.01760+0.006860*br+0.1732*(br**2)
#r.add_column(v,name="V")
r.write('gaia.csv',format='csv',overwrite="True")
%bell

UCAC 4

In [2]:
from astroquery.vizier import Vizier
v = Vizier(columns=['RAJ2000','DEJ2000','Vmag'], column_filters={"Vmag":"11..12"})
v.ROW_LIMIT = 10000000000000
coord = SkyCoord(ra = 0,dec = 90, unit = (u.deg,u.deg),frame='icrs')
result = v.query_region(coord, radius=45*u.deg, catalog='I/322A/out')
result[0].write('GSC_up.csv',format='csv',overwrite="True")

from astroquery.vizier import Vizier
v = Vizier(columns=['RAJ2000','DEJ2000','Vmag'], column_filters={"Vmag":"11..12"})
v.ROW_LIMIT = 10000000000000
coord = SkyCoord(ra = 0,dec = -90, unit = (u.deg,u.deg),frame='icrs')
result = v.query_region(coord, radius=45*u.deg, catalog='I/322A/out')
result[0].write('GSC_down.csv',format='csv',overwrite="True")

for i in range(8):
    v = Vizier(columns=['RAJ2000','DEJ2000','Vmag'], column_filters={"Vmag":"11..12"})
    v.ROW_LIMIT = 10000000000000
    coord = SkyCoord(ra = 22.5+45*(i+1),dec = 22.5, unit = (u.deg,u.deg),frame='icrs')
    result = v.query_region(coord, width=45*u.deg, catalog='I/322A/out')
    result[0].write('GSC_{}.csv'.format(i),format='csv',overwrite="True")
    
for i in range(8):
    v = Vizier(columns=['RAJ2000','DEJ2000','Vmag'], column_filters={"Vmag":"11..12"})
    v.ROW_LIMIT = 10000000000000
    coord = SkyCoord(ra = 22.5+45*(i+1),dec = -22.5, unit = (u.deg,u.deg),frame='icrs')
    result = v.query_region(coord, width=45*u.deg, catalog='I/322A/out')
    result[0].write('GSC_{}.csv'.format(i+8),format='csv',overwrite="True")
    

In [4]:
name_list = []
for file in glob.glob("GSC_*.csv"):
    name_list.append(file)
name_list.sort()
t = pd.concat([pd.read_csv(i) for i in name_list])
t.sort_values(by=['Vmag'], inplace=True)
t.to_csv('GSC.csv')

In [37]:
from astroquery.vo_conesearch import ConeSearch
coord = SkyCoord(ra = 0,dec = 90, unit = (u.deg,u.deg),frame='icrs')
result = ConeSearch.query_region(coord, '0.1 deg')

Bright Star Catalogue

In [16]:
from astroquery.vizier import Vizier
catalog_strs = Vizier.get_catalogs("V/50")
Vizier.ROW_LIMIT = -1
catalog_strs[0].write("BSC5P.csv",format="csv",overwrite="True")

Extract the bayer-flamsteed id

Star names.

Step 1: Download data from kaggle

In [124]:
kaggle.api.authenticate()

kaggle.api.dataset_download_files('ecotner/named-stars/IAU-CSN.csv', path='/home/sahyadri/Desktop/Hopping', unzip=True)

Step 2: Use pandas map fucntion to cross-map the names onto tycho data (LICH IS MATCHED INCORRECTLY!!!)
        

In [125]:
names_table = pd.read_csv('IAU-CSN.csv')
names_table = pd.DataFrame(names_table)

ttable = pd.read_csv('tycho_1.csv')
ttable = pd.DataFrame(ttable)
names_table["HIP"] = [x if x!='-' else 0 for x in names_table["HIP"]]

ttable.insert(1,"Name",np.zeros(len(ttable['TYC'])))

names_table["HIP"] = pd.to_numeric(names_table['HIP'],errors='coerce')

ttable['Name'] = ttable.HIP.map(names_table.set_index('HIP').Name)

ttable.to_csv('tycho_1.csv')



Cross Catalogue

In [74]:
v = Vizier(columns = ['HD','TYC','HIP','Vmag','Fl','Bayer','Cst'])
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
cross_catalog = v.get_catalogs('IV/27A/catalog')[0]
bayer_const = [x['Bayer']+' '+x['Cst'] for x in cross_catalog]
cross_catalog.remove_columns("Bayer")
cross_catalog['BayerConst'] = bayer_const
cross_catalog.write("CrossCatalog.csv",format='csv',overwrite="True")


(array([], dtype=int64),)


In [143]:
ccat = pd.read_csv('CrossCatalog.csv')
ccat = pd.DataFrame(ccat)
ccat["HIP"].fillna("0", inplace = True)



ttable = pd.read_csv('tycho_1.csv')
ttable = pd.DataFrame(ttable)

ccat = ccat.loc[~ccat.HIP.duplicated(keep='first')]

ttable.insert(4,"Bayer",np.zeros(len(ttable['_RAJ2000'])))

ttable['Bayer'] = ttable.HIP.map(ccat.set_index('HIP').BayerConst,na_action="ignore")

ttable.to_csv('tycho_1.csv')

ttable = Table.read('tycho_1.csv')
ttable.remove_columns(['col0','Unnamed: 0'])
ttable.write('tycho_1.csv',format='csv',overwrite="True")

KeyError: 'HIP: no such field'

Cross catalogue of NGC/M common names

In [20]:
v = Vizier(columns = ['Object','Name'])
v.ROW_LIMIT = -1
v.TIMEOUT = 1000
cross_catalog = v.get_catalogs('VII/118/names')[0]

cross_catalog['Name'] =  ['IC '+x.split()[1] if len(x.split())==2 and x.split()[0]=="I" else 'IC'+x.split()[0][1:] if len(x.split())==1 and x.split()[0]=="I" else 'NGC '+x for x in cross_catalog['Name']]

cross_catalog.write("DsCrossCatalog.csv",format='csv',overwrite="True")

In [23]:
ccat = pd.read_csv('DsCrossCatalog.csv')
ccat = pd.DataFrame(ccat)

ccat = ccat.loc[~ccat.Name.duplicated(keep='first')]

ntable= pd.read_csv('NGC.csv')
ntable = pd.DataFrame(ntable)

ntable.insert(2,"Common Name",np.zeros(len(ntable['_RAJ2000'])))

ntable['Common Name'] = ntable.Name.map(ccat.set_index('Name').Object,na_action="ignore")
ntable.to_csv('NGC.csv')

ntable = Table.read('NGC.csv')
ntable.remove_columns(['col0'])
ntable.write('NGC.csv',format='csv',overwrite="True")

Joining two tycho tables

In [47]:
t1 = Table.read('tycho_1.csv')
t2 = Table.read('tycho_2.csv')
t = vstack([t1, t2])
t.write('tycho_12_join.csv',format='csv',overwrite="True")

t1 = Table.read('tycho_12_join.csv')
t2 = Table.read('tycho_3.csv')
t = vstack([t1, t2])
t.write('tycho_123_join.csv',format='csv',overwrite="True")